In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("/home/rafael/git/projeto_bom_pagador/dataset/train.csv", index_col='ID')

In [3]:
df.rename(columns={"default payment next month":"default"}, inplace=True)

Criando datasets para consultas internas e externas.

In [4]:
consul_int, consul_ext = train_test_split(df, test_size=0.5)

consul_int.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_interno.csv")
consul_ext.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_externo.csv")

Trabalhando com o modelo de classificação

In [25]:
x = df.drop('default', axis=1)
y = df['default']

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [97]:
clf = RandomForestClassifier(max_depth=15, random_state=0)
clf.fit(x_train, y_train)
resul = clf.predict(x_test)
accuracy_score(y_test, resul)*100

81.9047619047619